In [ ]:
import scipy.misc 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
image = mpimg.imread('test_images/test1.jpg')
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)

In [ ]:
from camera_img_undistorter import CameraImageUndistorter
img_undistorter = CameraImageUndistorter()